In [22]:
# pip install kagglehub

In [23]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("rakeshkapilavai/extrovert-vs-introvert-behavior-data")

# print("Path to dataset files:", path)

# 외향형(Extrovert) vs. 내향형(Introvert) 구별 (이진 분류)
- HistGradientBoostingClassifier + GridSearchCV로 최적의 하이퍼파라미터 탐색

In [24]:
# 데이터 불러오기

import pandas as pd

df = pd.read_csv('personality_datasert.csv')
df.head()

,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,4.0,No,4.0,6.0,No,13.0,5.0,Extrovert
1,9.0,Yes,0.0,0.0,Yes,0.0,3.0,Introvert
2,9.0,Yes,1.0,2.0,Yes,5.0,2.0,Introvert
3,0.0,No,6.0,7.0,No,14.0,8.0,Extrovert
4,3.0,No,9.0,4.0,No,8.0,5.0,Extrovert


In [30]:
# 전처리

yes_no = {'No': 0, 'Yes': 1}
ex_in = {'Introvert': 0, 'Extrovert': 1}

# df = df.replace(to_replace={'Stage_fear': {'No': 0, 'Yes': 1}})
# df = df.replace(to_replace={'Drained_after_socializing': {'No': 0, 'Yes': 1}})
df = df.replace(to_replace={'Stage_fear': yes_no, 'Drained_after_socializing': yes_no})
df = df.replace(to_replace={'Personality': ex_in})

df.head()

,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,4.0,0,4.0,6.0,0,13.0,5.0,1
1,9.0,1,0.0,0.0,1,0.0,3.0,0
2,9.0,1,1.0,2.0,1,5.0,2.0,0
3,0.0,0,6.0,7.0,0,14.0,8.0,1
4,3.0,0,9.0,4.0,0,8.0,5.0,1


In [36]:
# 학습 데이터와 타겟(Personality) 데이터 분리

personality_target = df['Personality'].to_numpy()  # 타겟은 Personality
personality_data = df.drop('Personality', axis=1).to_numpy()  # axis=1: 열 기준에서 Personality를 제외

'''
위의 코드는 아래와 같은 의미의 코드다.

personality_target = df['Personality'].to_numpy()
personality_data = df[['Time_spent_Alone', 'Stage_fear', 'Social_event_attendance', 'Going_outside', 'Drained_after_socializing', 'Friends_circle_size', 'Post_frequency']].to_numpy()
'''

print("personality_target:\n", personality_target, "\n")
print("personality_data:\n", personality_data)

personality_target:
 [1 0 0 ... 0 0 1] 

personality_data:
 [[ 4.  0.  4. ...  0. 13.  5.]
 [ 9.  1.  0. ...  1.  0.  3.]
 [ 9.  1.  1. ...  1.  5.  2.]
 ...
 [ 4.  1.  1. ...  1.  4.  0.]
 [11.  1.  1. ...  1.  2.  0.]
 [ 3.  0.  6. ...  0.  6.  9.]]


In [38]:
# 훈련 세트와 테스트 세트 분리

from sklearn.model_selection import train_test_split, GridSearchCV

train_input, test_input, train_target, test_target = train_test_split(
    personality_data, personality_target, test_size=0.1
)

In [45]:
# 최적의 모델 찾기!
# - HistGradientBoostingClassfier

from sklearn.ensemble import HistGradientBoostingClassifier
import numpy as np
from sklearn.model_selection import cross_validate

params = {
    'max_iter': np.arange(90, 300, 10),
    'learning_rate': [0.0001, 0.001, 0.002, 0.003, 0.004, 0.005, 0.01], # 가중치를 저장하는 비율
}

gs = GridSearchCV(HistGradientBoostingClassifier(), params, n_jobs=-1)
gs.fit(train_input, train_target)  # 학습

print(gs.cv_results_['mean_test_score'])  # 각 파라미터 조합의 평균 정확도
print(gs.best_params_)                    # 최고 성능 파라미터

hgb = gs.best_estimator_

print("학습 세트 정확도:\n", hgb.score(train_input, train_target))
print("테스트 세트 정확도:\n", hgb.score(test_input, test_target))

[0.50842912 0.50842912 0.50842912 0.50842912 0.50842912 0.50842912
 0.50842912 0.50842912 0.58965517 0.73869732 0.80038314 0.82375479
 0.83869732 0.84597701 0.87394636 0.88467433 0.89157088 0.89463602
 0.89578544 0.89770115 0.89808429 0.93409962 0.93409962 0.93409962
 0.93524904 0.93524904 0.93524904 0.93524904 0.93524904 0.93524904
 0.93524904 0.93524904 0.93524904 0.93524904 0.93524904 0.93524904
 0.93524904 0.93524904 0.93524904 0.93524904 0.93524904 0.93524904
 0.93524904 0.93524904 0.93524904 0.93524904 0.93524904 0.93524904
 0.93524904 0.93524904 0.93524904 0.93524904 0.93524904 0.93524904
 0.93524904 0.93524904 0.93524904 0.93524904 0.93524904 0.93524904
 0.93524904 0.93524904 0.93524904 0.93524904 0.93524904 0.93524904
 0.93524904 0.93524904 0.93524904 0.93524904 0.93524904 0.93524904
 0.93524904 0.93524904 0.93524904 0.93524904 0.93524904 0.93524904
 0.93524904 0.93524904 0.93524904 0.93524904 0.93524904 0.93524904
 0.93524904 0.93524904 0.93524904 0.93524904 0.93524904 0.9352

In [44]:
# 학습한 최적의 모델 저장

import pickle
with open("model.pkl", "wb") as f:
    pickle.dump(hgb, f)

personality_data[:5]

array([[ 4.,  0.,  4.,  6.,  0., 13.,  5.],
       [ 9.,  1.,  0.,  0.,  1.,  0.,  3.],
       [ 9.,  1.,  1.,  2.,  1.,  5.,  2.],
       [ 0.,  0.,  6.,  7.,  0., 14.,  8.],
       [ 3.,  0.,  9.,  4.,  0.,  8.,  5.]])